## Confidence interval

- Suppose we have a hypothesis `H`. This could be anything, but simplicity, let's consider `H` to refer to the mean of the population
    - 
    - When talking about CI, we always speak of CI w.r.t some confidence level $\alpha$
    - 